<a href="https://colab.research.google.com/github/flaviohds/dota_analysis/blob/main/dota_update_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time

In [ ]:
!pip install selenium # mimics user clicks
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver #install chrome on the scrapper server

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

#chrome options that reduces bugs and variability
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36')
chrome_options.add_argument('--ignore-certificate-errors')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options, service_args=['--verbose', '--log-path=/tmp/chromedriver.log'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') #mount drive on google drive

import os
os.chdir("drive/My Drive/dota") #map the folder on google drive

In [ ]:
#function updates heroes names from www.dota2.com/heroes
def update_hero_list():
  print('updating hero names')
  print('expected time: ~3mins 20s')
  wd.delete_all_cookies()
  wd.get("https://www.dota2.com/hero/abaddon")
  wd.page_source;
  time.sleep(2)
  
  herolist=[]
  i=0
  while i!=1:
    hero_name=wd.find_element('xpath','/html/body/div[2]/div/div/div[2]/div[5]/div[2]')
    
    if hero_name.text=="ZEUS":
      i=1
    herolist+=[hero_name.text]



    next_hero_b=wd.find_element('xpath','/html/body/div[2]/div/div/div[2]/div[2]/a[3]')
    next_hero_b.click()
    time.sleep(1)



  df=pd.DataFrame(herolist,columns=["HERO_NAME"])
  df.to_excel("hero_list.xlsx")

In [ ]:
#function updates hero win and pickrates from www.dotabuff.com
def update_win_pick():
  print('updating win and pick rates')
  print('expected time: ~1min')
  wd.delete_all_cookies()
  wd.get("https://www.dotabuff.com/heroes/trends")
  wd.page_source;
  time.sleep(2)

  data_coord=[2,3,4,6,7,8]
  data=[]

  hero_df=pd.read_excel('hero_list.xlsx',usecols=[1])
  total_heroes=hero_df.shape[0]

  order_name_b=wd.find_element('xpath','/html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/thead/tr[3]/th[1]') #
  order_name_b.click()  #orders table by hero name

  for j in range(total_heroes):
    data_row=[]
    for i in range(len(data_coord)):
      xpath='/html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr['+str(j+1)+']/td['+str(data_coord[i])+']'
      html_element=wd.find_element('xpath',xpath)
      try:
        data_row+=[float(html_element.text.replace('%','').replace('+',''))] #removes % and + signs, converts to float and adds to the list
      except:
        data_row+=[0]

    data.append(data_row)

  hero_df=pd.read_excel('hero_list.xlsx',usecols=[1])
  hero_df[['WIN_RATE_OLD','WIN_RATE','WIN_RATE_VAR','PICK_RATE_OLD','PICK_RATE','PICK_RATE_VAR']]=data


  # xpaths. last checked 23 june 2022
  # first hero
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[2]  old_win
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[3]  new_win
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[4]  change_win
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[6]  old_pick
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[7]  new_pick
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[8]  change_pick
  # second hero
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[2]/td[2]  ''
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[2]/td[3]  ''

  hero_df.to_excel("database.xlsx")

In [ ]:
update_hero_list()
update_win_pick()

In [ ]:
#to take a screenshot from chrome

# from PIL import Image
# from io import BytesIO

# figure=wd.get_screenshot_as_png()
# im = Image.open(BytesIO(figure))
# im